In [10]:
import os
from skimage import io
import numpy as np
import cv2
from scipy import ndimage
import numpy as np

def calc_orien(x):
    row = np.zeros((len(x), 2), dtype=np.float64)
    for i in range(row.shape[0]):
        row[i, 0] = x[i, 0, 0]
        row[i, 1] = x[i, 0, 1]

    centroid = np.mean(row, axis=0)
    row -= centroid
    cov_mat = np.cov(row.T)
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    angle = np.arctan2(eig_vecs[0, 1], eig_vecs[0, 0])
    return angle

dataset_dir = 'KIMIA dataset'

template_image = cv2.imread('Templete_Input.png')
template_image = cv2.cvtColor(template_image, cv2.COLOR_BGR2GRAY)
_, template_image = cv2.threshold(template_image, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
contours, _ = cv2.findContours(template_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for i, contour in enumerate(contours):
    ar = cv2.contourArea(contour)
    if ar >= 1e3:
        angle_template = calc_orien(contour)
        break

image_paths = []
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.png'):
            image_paths.append(os.path.join(root, file))

image_names = [os.path.basename(path) for path in image_paths]
aligned_image_names = [name.replace('.png', '_aligned.png') for name in image_names]

aligned_images = []
for image_path in image_paths:
    image = cv2.imread(image_path)
    image_1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image_1 = cv2.threshold(image_1, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(image_1, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for i, contour in enumerate(contours):
        ar = cv2.contourArea(contour)
        if ar >= 1e3:
            angle_image = calc_orien(contour)
            break
    
    rotation_angle = angle_image - angle_template
    rotation_angle = np.rad2deg(rotation_angle)

    h, w = image.shape[:2]
    center = (w // 2, h // 2)

    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    aligned_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    aligned_images.append(aligned_image)

output = "KIMIA Aligned"
os.makedirs(output, exist_ok=True)

for i, image in enumerate(aligned_images):
    image_name = aligned_image_names[i]
    image_path = os.path.join(output, image_name)
    cv2.imwrite(image_path, image)

## Flowchart

Start
|
|-- Load Template Image
|   |
|   |-- Convert to Grayscale
|   |
|   |-- Threshold Image
|   |
|   |-- Calculate Orientation (angle_template)
|
|-- Iterate Through Images in "KIMIA dataset"
|   |
|   |-- Load Current Image
|   |   |
|   |   |-- Convert to Grayscale
|   |   |
|   |   |-- Threshold Image
|   |   |
|   |   |-- Calculate Orientation (angle_image)
|   |
|   |-- Calculate Rotation Angle (angle_image - angle_template)
|   |
|   |-- Is Rotation Angle Significant? (angle_image - angle_template >= threshold)
|   |   |
|   |   |-- Yes
|   |   |   |
|   |   |   |-- Create Rotation Matrix
|   |   |   |
|   |   |   |-- Apply Rotation to Image (Image Alignment)
|   |   |   |
|   |   |   |-- Save Aligned Image
|   |   |
|   |   |-- No
|   |   |   |
|   |   |   |-- Skip Image
|
|-- End
